In [207]:
import psi4 
import numpy as np

In [208]:
# ==> Opciones Básicas Psi4 <==
# Memoria
psi4.set_memory(int(5e8))
numpy_memory = 700

# Output
psi4.core.set_output_file('output.dat', False)

# Geometría
mol = psi4.geometry("""
0 1
   C       -4.69625        3.24199        0.00000
   H       -3.62625        3.24199        0.00000
   H       -5.05292        4.22553       -0.22434
   H       -5.05292        2.94449        0.96394
   H       -5.05292        2.55593       -0.73961
""")

# Opciones de calculo
psi4.set_options({'basis': 'cc-pvtz',
                  'e_convergence': 1e-8})

psi4.energy('scf/cc-pvtz')

-40.213062902362196

In [209]:
# Máximo de iteraciones SCF
MAXITER = 40
# Criterio de Convergencia para la energía.
E_conv = 1.0e-8
# Energía de repulsión nucelo-nucleo
E_nuc = mol.nuclear_repulsion_energy()

In [210]:
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))

In [211]:
#posee todas las integrales posibles dentro del set de bases
mints = psi4.core.MintsHelper(wfn.basisset())

In [212]:
S = np.asarray(mints.ao_overlap())


In [213]:
#Hamiltoniano de core 
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V



In [214]:
# S.shape = (número de cloumnas, número de filas)
nbf = S.shape
print nbf
print nbf[0]
ndocc = wfn.nalpha()
print ndocc 

(86, 86)
86
5


In [215]:
# Memory check for ERI tensor
I_size = (nbf[0]**4) * 8.e-6
print('\nSize of the ERI tensor will be {:4.2f} MB.'.format(I_size))
memory_footprint = I_size * 1.5


Size of the ERI tensor will be 437.61 MB.


In [216]:
# Tensor con integrales 2e-
I = np.asarray(mints.ao_eri())
print I.shape

(86, 86, 86, 86)


In [217]:
e1 = np.linalg.eig(S)[0]
U = np.linalg.eig(S)[1]
U_dag = np.linalg.inv(U)

Zeros= np.identity(len(S))
s_sqrt = Zeros*np.sqrt(e1)

s_sqrt_inv = np.linalg.inv(s_sqrt)


In [218]:
#Equivalente a # ==> Construct AO orthogonalization matrix A <==
#A = mints.ao_overlap()
#A.power(-0.5, 1.e-16)
#A = np.asarray(A)

S_sqrt_inv = U.dot(s_sqrt_inv).dot(U_dag)

In [219]:
# Check orthonormality
S_p = S_sqrt_inv.dot(S).dot(S_sqrt_inv)
new_hope = np.allclose(S_p, np.eye(S.shape[0]))

if new_hope:
    print('There is a new hope for diagonalization!')
else:
    print("Whoops...something went wrong. Check that you've correctly built the transformation matrix.")
    

There is a new hope for diagonalization!


In [220]:
# ==> Compute C & D matrices with CORE guess <==
# Transformed Fock matrix
F_p = S_sqrt_inv.dot(H).dot(S_sqrt_inv)

# Diagonalize F_p for eigenvalues & eigenvectors with NumPy
e, C_p = np.linalg.eigh(F_p)

# Transform C_p back into AO basis
C = S_sqrt_inv.dot(C_p)

# Grab occupied orbitals
C_occ = C[:, :ndocc]

# Build density matrix from occupied orbitals
D = np.einsum('pi,qi->pq', C_occ, C_occ)

# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()

# ==> SCF Iterations <==
# Pre-iteration energy declarations
SCF_E = 0.0
E_old = 0.0

print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrix
    J = np.einsum('pqrs,rs->pq', I, D)
    K = np.einsum('prqs,rs->pq', I, D)
    F = H + 2*J - K
    
    # Compute RHF energy
    SCF_E = np.einsum('pq,pq->', (H + F), D) + E_nuc
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E' % (scf_iter, SCF_E, SCF_E - E_old))
    
    # SCF Converged?
    if (abs(SCF_E - E_old) < E_conv):
        break
    E_old = SCF_E
    
    # Compute new orbital guess
    F_p =  S_sqrt_inv.dot(F).dot(S_sqrt_inv)
    e, C_p = np.linalg.eigh(F_p)
    C = S_sqrt_inv.dot(C_p)
    C_occ = C[:, :ndocc]
    D = np.einsum('pi,qi->pq', C_occ, C_occ)
    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        psi4.core.clean()
        raise Exception("Maximum number of SCF iterations exceeded.")

# Post iterations
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % (SCF_E))

==> Starting SCF Iterations <==

SCF Iteration   1: Energy = -27.1315902792864669 dE = -2.71316E+01
SCF Iteration   2: Energy = -33.9843286561506517 dE = -6.85274E+00
SCF Iteration   3: Energy = -37.3232698652068251 dE = -3.33894E+00
SCF Iteration   4: Energy = -39.2425850406818455 dE = -1.91932E+00
SCF Iteration   5: Energy = -39.9706647654681717 dE = -7.28080E-01
SCF Iteration   6: Energy = -40.1694574362586536 dE = -1.98793E-01
SCF Iteration   7: Energy = -40.2056317075975329 dE = -3.61743E-02
SCF Iteration   8: Energy = -40.2118366179224296 dE = -6.20491E-03
SCF Iteration   9: Energy = -40.2128587052404214 dE = -1.02209E-03
SCF Iteration  10: Energy = -40.2130271718755665 dE = -1.68467E-04
SCF Iteration  11: Energy = -40.2130548747750254 dE = -2.77029E-05
SCF Iteration  12: Energy = -40.2130594351409769 dE = -4.56037E-06
SCF Iteration  13: Energy = -40.2130601857521270 dE = -7.50611E-07
SCF Iteration  14: Energy = -40.2130603093274317 dE = -1.23575E-07
SCF Iteration  15: Energy = -